# Polygonization

In this workshop we will learn how to convert our topological model of the building into a geometric model.

In [ ]:
import topogenesis as tg
import numpy as np 
import pyvista as pv
import os
import copy
import trimesh as tm

In [ ]:
# create all possible configurations
l_bis = []
for i in range(2**8):
    bi = np.array(list(np.binary_repr(i, width=8))).astype(int).reshape((2,2,2))
    l_bi = tg.to_lattice(bi, [0,0,0])
    l_bis.append(l_bi)

In [ ]:
#  creating stencils
so_base = tg.create_stencil("von_neumann", 0, 1)
sx_base = copy.deepcopy(so_base)
sx_base.set_index((0,0,0), 0)
sx_base.set_index((1,0,0), 1)

# setting separate direction
corner_state = dict(
    OO = so_base,
    XP = sx_base,
    XN = np.flip(sx_base, 0),
    YP = np.transpose(sx_base, (1,0,2)),
    YN = np.transpose(np.flip(sx_base, 0), (1,0,2)),
    ZP = np.transpose(sx_base, (2,1,0)),
    ZN = np.transpose(np.flip(sx_base, 0), (2,1,0))
)
# setting paired directions
corner_state["XX"] = corner_state["XN"] + corner_state["XP"]
corner_state["YY"] = corner_state["YN"] + corner_state["YP"]
corner_state["ZZ"] = corner_state["ZN"] + corner_state["ZP"]

# dummy symmetry string 
# sym_str = [["OO"], ["XP"], ["XN"], ["YP"], ["YN"], ["ZP"], ["ZN"]]
# sym_str = [["OO"], ["XX"], ["YP"], ["YN"], ["ZP"], ["ZN"]]
sym_str = [["OO"], ["XX", "YY"], ["ZP"], ["ZN"]]
# sym_str = [["OO"], ["XX", "YY"], ["ZZ"]]
# sym_str = [["OO"], ["XX", "YY", "ZZ"]]
# sym_str = [["OO"]]

stencils = []

for i, sym in enumerate(sym_str):
    # init the symmetry stencil with the first item
    sym_stencil = copy.deepcopy(corner_state[sym[0]])
    # iteratively add the rest of the symmetry stencils together
    for key in sym[1:]:
        sym_stencil += corner_state[key]
    # append
    stencils.append(sym_stencil)
    # compute the sum of all stencils
    if i == 0:
        stencil_sum = copy.deepcopy(sym_stencil)
    else:
        stencil_sum += sym_stencil

# check if there is any neighbours that are not included
stencil_rest = tg.create_stencil("von_neumann", 1, 1) - stencil_sum

# print(np.zeros((3,3,3)).sum())
if (np.array(stencil_rest).sum()):
    stencil_rest.function = tg.sfunc.sum
    stencils.append(stencil_rest)

# add the sum function to all stencils
for s in stencils:
    s.function = tg.sfunc.sum

In [ ]:
results = []
for l in l_bis:
    # apply the stencils in all corners of all possible configurations
    r_s = [l.apply_stencil(s).flatten() for s in stencils]

    # concatenate results into columns
    result = np.c_[tuple(r_s)]

    results.append(result)

# stack results vertically
res_stacked = np.vstack(results)

# find the uniqe arangements of corners
uniq_corner_arang = np.unique(res_stacked, axis = 0)
print(uniq_corner_arang)
print(len(uniq_corner_arang))

In [ ]:
stencil_ind = []
for s in stencils:
    filled_ind = np.array(np.where(np.array(s)==1)).T
    #relocate to corner
    filled_ind_reloc = filled_ind - 1
    # flip vertically
    filled_ind_flpd = np.flip(filled_ind_reloc, 0).astype(int)

    stencil_ind.append(filled_ind_flpd)
print(stencil_ind)

In [ ]:
# generate mini-lattices to represent the geometrical equivalent of unique corner arrangments
base_zero = np.zeros((2,2,2), dtype=np.int8)

corner_neigh_lattices = []
corner_loc_lattices = []
# for each unique arangement, create a representation
for uca in uniq_corner_arang:
    corner_arang = np.copy(base_zero)
    corner_loc = np.copy(base_zero)
    corner_loc[0,0,0] = 1

    shift = np.zeros(3)
    # for each list of indices of each stencil
    for i, s_inds in enumerate(stencil_ind):
        # for each index
        for j, s_i in enumerate(s_inds):
            # check if the sum is big enough to mark this index
            if uca[i] > j:
                # check if the index is positive
                if s_i.sum() >= 0:
                    # mark as one
                    corner_arang[tuple(s_i)] = 1
                # if the index is negative
                else:
                    # mark as one
                    corner_arang[tuple(s_i)] = 1
                    # record the index as a shift
                    shift += s_i

    # roll nwighbourhood and locations
    corner_arang = np.roll(corner_arang, tuple(shift.astype(int)), (0,1,2))
    corner_loc = np.roll(corner_loc, tuple(shift.astype(int)), (0,1,2))

    # convert to lattice
    corner_lat = tg.to_lattice(corner_arang, np.zeros(3))
    corner_neigh_lattices.append(corner_lat)

    corner_loc_lat = tg.to_lattice(corner_loc, np.zeros(3))
    corner_loc_lattices.append(corner_loc_lat)

In [ ]:
p = pv.Plotter(notebook=True)

base_lattice = corner_neigh_lattices[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# # adding the avilability lattice
# init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):
    f = int(value)
    lattice = corner_neigh_lattices[f]
    loc = corner_loc_lattices[f]

    # Add the data values to the cell data
    grid.cell_arrays["filled"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([.9, 1.1], scalars="filled")
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=0.7, show_scalar_bar=False)
    
    # Add the data values to the cell data
    grid.cell_arrays["corner"] = loc.flatten(order="F").astype(int)# Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([.9, 1.1], scalars="corner")
    # adding the voxels
    p.add_mesh(threshed, name='sphere2', show_edges=True, opacity=1.0, show_scalar_bar=False, color="white")

    return

p.add_slider_widget(create_mesh, [1, len(corner_neigh_lattices)], title='Arrangements', value=1, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)

In [ ]:
def to_csv(lattice, filepath: str):
        """This method saves the lattice to a csv file

        Args:
            filepath: path to the csv file
        """
        # volume to panda dataframe
        vol_df = lattice.to_panadas()

        # specifying metadata and transposing it
        metadata = pd.DataFrame({
            'minbound': lattice.minbound,
            'shape': np.array(lattice.shape),
            'unit': lattice.unit,
        })

        with open(filepath, 'w') as df_out:

            metadata.to_csv(df_out, index=False,header=True, float_format='%g', line_terminator='n')

            df_out.write('\n')

            vol_df.to_csv(df_out, index=False, float_format='%g')

In [123]:
interior_zero = tg.to_lattice(np.zeros((2,2,2), dtype=np.int8), [0,0,0])
border_pad = np.pad(interior_zero, (1,1), 'constant', constant_values=(1))
base_zero = tg.to_lattice(np.zeros((4,4,4), dtype=np.int8), [0,0,0])
s = tg.create_stencil("von_neumann", 1,1)
s.set_index([0,0,0], 0)

for i, core, neigh in zip(range(len(corner_loc_lattices)), corner_loc_lattices, corner_neigh_lattices):
    # construct saving paths
    core_path = os.path.relpath('../data/bmc_templates/core_' + f'{i:02}' + '.csv')
    neigh_path = os.path.relpath('../data/bmc_templates/neighbourhood_' + f'{i:02}' + '.csv')

    # construct the padded core
    core_pad = tg.to_lattice(np.pad(core, (1,1), 'constant', constant_values=(0)), [0,0,0])
    # extract the outer neighbours of the core
    core_3ind = np.array(np.where(core_pad==1)).flatten()
    pals = base_zero.find_neighbours_masked(s, loc=core_3ind)
    extra_neighs = np.copy(base_zero).flatten()
    extra_neighs[pals] = 1
    extra_neighs = tg.to_lattice(extra_neighs.reshape(4,4,4), [0,0,0])
    # remove interior pals
    extra_neighs *= border_pad

    # aggregate neighbours
    neigh = np.pad(neigh, (1,1), 'constant', constant_values=(0)) + extra_neighs

    print(type(core_pad))
    print(type(neigh))
    break



<class 'topogenesis.datastructures.datastructures.lattice'>
<class 'topogenesis.datastructures.datastructures.lattice'>


In [97]:
lattice = corner_neigh_lattices[4]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = lattice.minbound - lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = lattice.unit 

# Add the data values to the cell data
grid.cell_arrays["filled"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([.9, 1.1], scalars="filled")
# extract mesh
mesh = threshed.extract_surface()
points = mesh.points
quad_faces = mesh.faces.reshape((-1, 5))[:, 1:]
tri_faces = np.c_[quad_faces[:,0:3], quad_faces[:,2:4], quad_faces[:,0]].reshape((-1, 3))
tri_faces_pv = np.c_[np.full(len(tri_faces), 3), tri_faces]
# pv.PolyData(points, tri_faces)
tri_mesh = tm.Trimesh(points, tri_faces)
# out = 
# out

with open(obj_path, 'w') as file:
    file.write(tm.exchange.obj.export_obj(tri_mesh))
# tri_mesh.show()
# tri_faces_pv
# tri_faces

### Credits

In [ ]:
__aut__author__ = "Shervin Azadi"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Polygonization"